In [1]:
import cv2
import numpy as np


In [5]:
# Read the images to be aligned
im1 =  cv2.imread('/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw_frames/scare_clips/PZ/APR05_2018_I_DJI_0012/APR05_2018_I_DJI_0012_11342.jpg' )
im2 =  cv2.imread('/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw_frames/scare_clips/PZ/APR05_2018_I_DJI_0012/APR05_2018_I_DJI_0012_11344.jpg' )

root  = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw_frames/scare_clips/PZ/APR05_2018_I_DJI_0012/APR05_2018_I_DJI_0012_113'
    
ims_file = [root + str(36 + ind*2) + '.jpg' for ind in range(10)] 
ims = [cv2.imread(file) for file in ims_file]

down_sampling = 2
    
new_shape = (int(ims[0].shape[1]/down_sampling), int(ims[0].shape[0]/down_sampling)) 

# im1 = cv2.resize(im1, new_shape, interpolation=cv2.INTER_LINEAR)
# im2 = cv2.resize(im2, new_shape, interpolation=cv2.INTER_LINEAR)

ims_small = [cv2.resize(im, new_shape, interpolation=cv2.INTER_LINEAR) for im in ims]

im_shape = ims_small[0].shape



# im1 = im1[:, width_start:-width_start]
# im2 = im2[:, width_start:-width_start]



# # Convert images to grayscale
# im1_gray = cv2.cvtColor(im1,cv2.COLOR_BGR2GRAY)
# im2_gray = cv2.cvtColor(im2,cv2.COLOR_BGR2GRAY)

ims_green = [im[:, :, :] for im in ims_small]
# # Find size of image1
# sz = im1.shape
 
# # Define the motion model
# warp_mode = cv2.MOTION_EUCLIDEAN
 
# # Define 2x3 or 3x3 matrices and initialize the matrix to identity
# if warp_mode == cv2.MOTION_HOMOGRAPHY :
#     warp_matrix = np.eye(3, 3, dtype=np.float32)
# else :
#     warp_matrix = np.eye(2, 3, dtype=np.float32)
 
#  # Specify the number of iterations.
# number_of_iterations = 3000;
 
# # Specify the threshold of the increment
# # in the correlation coefficient between two iterations
# termination_eps = 1e-10;
 
# # Define termination criteria
# criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations,  termination_eps)

warps = []
aligned = []
total_warp = np.eye(3)



for im_ind, im in enumerate(ims_green[:-1]):
    warp_matrix = cv2.estimateRigidTransform(im, ims_green[im_ind + 1], fullAffine=False) 
    aligned.append(cv2.warpAffine(ims_green[im_ind + 1], warp_matrix, (im_shape[1], im_shape[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP))
    warps.append(warp_matrix)
    warp_matrix = np.vstack((warp_matrix, np.array([0,0,1]))) 
    total_warp = np.matmul(total_warp, warp_matrix)
    
total_warp = total_warp[:2,]



In [6]:
total_warp

array([[   0.9568    ,   -0.29311721,  202.80828144],
       [   0.29311721,    0.9568    , -281.61435939]])

array([[   0.95607778,   -0.29434366,  183.41945556],
       [   0.29434366,    0.95607778, -140.99255209]])

In [7]:
last_im = ims_green[-1]
for warp in warps[::-1]:
    last_im = cv2.warpAffine(last_im, warp, (im_shape[1], im_shape[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
second_go = cv2.warpAffine(ims_green[-1], total_warp, (im_shape[1], im_shape[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
cv2.imshow("Aligned Image 2 ", ims_green[0] - second_go)
cv2.imshow("im0", ims_green[0])
cv2.imshow("econd_go", second_go)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
# Run the ECC algorithm. The results are stored in warp_matrix.
(cc, warp_matrix) = cv2.findTransformECC (im1_gray,im2_gray,warp_matrix, warp_mode, criteria)
print(cc)
 
if warp_mode == cv2.MOTION_HOMOGRAPHY :
    # Use warpPerspective for Homography 
    im2_aligned = cv2.warpPerspective (im2, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
else :
    # Use warpAffine for Translation, Euclidean and Affine
    im2_aligned = cv2.warpAffine(im2, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP);

0.996518496808582


In [4]:
 # Show final results
cv2.imshow("Image 1", im1 - im2)
cv2.imshow("Image 2", im1 - im2_aligned)
cv2.imshow("Aligned Image 2", im2_aligned)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
warp_matrix = cv2.estimateRigidTransform(im1_gray, im2_gray, fullAffine=False) 
im2_aligned_2 = cv2.warpAffine(im2, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP);

0
10
20
30
40
50
60
70
80
90
0.5363452434539795


In [ ]:
 # Show final results
cv2.imshow("Image 1", im1 - im2)
cv2.imshow("Image 2", im1 - im2_aligned)
cv2.imshow("Aligned Image 2", im2_aligned)
cv2.imshow("Image 1a", im1 - im2)
cv2.imshow("Image 2a", im1 - im2_aligned_2)
cv2.imshow("Aligned Image 2a", im2_aligned_2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
warp_matrix

array([[  0.9995621 ,  -0.03279349,  17.95482768],
       [  0.03279349,   0.9995621 , -18.16499636]])

In [7]:
print(np.arccos(warp_matrix[0,0]))
print(np.arcsin(warp_matrix[1,0]))

0.029595028523110984
0.03279937003053213
